<a href="https://colab.research.google.com/github/yakaralar/IE492/blob/master/promotion_day_exclusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import os
import glob
import shutil
os.listdir()

['.config', 'drive', 'sample_data']

In [4]:
os.listdir("drive/My Drive/colab_datas/IE 492")

['Comparison of methods.pdf',
 '1-s2.0-S1364032114000914-main.pdf',
 '1-s2.0-S0360544209002539-main.pdf',
 '1-s2.0-S1876610211043116-main.pdf',
 '1-s2.0-S0305048311001423-main.pdf',
 'weather_turkey_processed.zip',
 'Meta_L_makaleler',
 'all_data.feather',
 'UECM-15012015-20022020.csv',
 'RealTimeConsumption-01122014-09032020.csv',
 'ozel_gunler',
 'RealTimeConsumption_unholidayized.csv']

In [5]:
shutil.copy("drive/My Drive/colab_datas/IE 492/RealTimeConsumption-01122014-09032020.csv","./")

'./RealTimeConsumption-01122014-09032020.csv'

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
%matplotlib inline
from tqdm import tqdm_notebook as tqdm

## Read cons df

In [7]:
# bruuuuuuuh
consumption_df=pd.read_csv("RealTimeConsumption-01122014-09032020.csv",encoding='latin-1')
consumption_df.columns=["Date","Hour","Consumption"]
consumption_df["Datetime"]=consumption_df["Date"]+" "+consumption_df["Hour"]
consumption_df["Consumption"]=consumption_df["Consumption"].apply(lambda x:x.replace(",","")).astype(float)
consumption_df["Datetime"]=pd.to_datetime(consumption_df["Datetime"])
consumption_df

,Date,Hour,Consumption,Datetime
0,31.12.2015,00:00,29590.88,2015-12-31 00:00:00
1,31.12.2015,01:00,27785.43,2015-12-31 01:00:00
2,31.12.2015,02:00,26516.53,2015-12-31 02:00:00
3,31.12.2015,03:00,26091.62,2015-12-31 03:00:00
4,31.12.2015,04:00,25871.54,2015-12-31 04:00:00
...,...,...,...,...
36715,09.03.2020,19:00,0.00,2020-09-03 19:00:00
36716,09.03.2020,20:00,0.00,2020-09-03 20:00:00
36717,09.03.2020,21:00,0.00,2020-09-03 21:00:00
36718,09.03.2020,22:00,0.00,2020-09-03 22:00:00


In [8]:
consumption_df=consumption_df.loc[consumption_df["Datetime"]<pd.to_datetime("2020-03-04 00:00:00")]
consumption_df

,Date,Hour,Consumption,Datetime
0,31.12.2015,00:00,29590.88,2015-12-31 00:00:00
1,31.12.2015,01:00,27785.43,2015-12-31 01:00:00
2,31.12.2015,02:00,26516.53,2015-12-31 02:00:00
3,31.12.2015,03:00,26091.62,2015-12-31 03:00:00
4,31.12.2015,04:00,25871.54,2015-12-31 04:00:00
...,...,...,...,...
36571,03.03.2020,19:00,37936.79,2020-03-03 19:00:00
36572,03.03.2020,20:00,37553.37,2020-03-03 20:00:00
36573,03.03.2020,21:00,36581.73,2020-03-03 21:00:00
36574,03.03.2020,22:00,35434.36,2020-03-03 22:00:00


## Datelar promosyon tipleri koyulacak, sonradan lineer regresyon direkt vermeme gerek yok 

In [9]:
special_days_path="drive/My Drive/colab_datas/IE 492/ozel_gunler/*"
special_day_df=pd.DataFrame()
exception=[]
for i in tqdm(glob.glob(special_days_path)):  
    a=pd.read_csv(i)
    try:
        a["special_day"]=np.full(len(a),a.columns[0])
        a.columns=["date","special_day"]
        special_day_df=pd.concat([special_day_df, a])
    except ValueError:
        exception.append(i)
        pass
special_day_df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


,date,special_day
1,2015-10-01,elections
2,2015-07-07,elections
3,2017-04-16,elections
4,2018-06-24,elections
5,2019-03-31,elections
...,...,...
6,2015-08-30,zaferbayrami
7,2016-08-30,zaferbayrami
8,2017-08-30,zaferbayrami
9,2018-08-30,zaferbayrami


In [0]:
special_day_df["is"]=np.ones(len(special_day_df))
special_day_df=pd.pivot_table(special_day_df,index="date",columns="special_day").fillna(0).astype(int)
special_day_df.columns=['_'.join(i) for i in special_day_df.columns]
special_day_df=special_day_df.reset_index()



## LR Date categorical and spec days


In [11]:
cons_lagged=consumption_df.copy()
cons_lagged["Consumption_2_lag"]=cons_lagged["Consumption"].shift(24*2)
cons_lagged["Consumption_7_lag"]=cons_lagged["Consumption"].shift(24*7)
cons_lagged["Hour"]=cons_lagged["Hour"].apply(lambda x:x.split(":")[0]).astype(int)
cons_lagged["Day"]=cons_lagged["Date"].apply(lambda x:x.split(".")[0]).astype(int)
cons_lagged["Month"]=cons_lagged["Date"].apply(lambda x:x.split(".")[1]).astype(int)
cons_lagged["weekday"]=pd.to_datetime(cons_lagged["Date"]).apply(lambda x: x.weekday()).astype(int)
cons_lagged=cons_lagged.dropna()
cons_lagged

,Date,Hour,Consumption,Datetime,Consumption_2_lag,Consumption_7_lag,Day,Month,weekday
168,07.01.2016,0,28763.95,2016-07-01 00:00:00,29244.38,29590.88,7,1,4
169,07.01.2016,1,27284.84,2016-07-01 01:00:00,27637.23,27785.43,7,1,4
170,07.01.2016,2,26321.95,2016-07-01 02:00:00,26335.78,26516.53,7,1,4
171,07.01.2016,3,25748.49,2016-07-01 03:00:00,25870.23,26091.62,7,1,4
172,07.01.2016,4,25636.58,2016-07-01 04:00:00,25975.07,25871.54,7,1,4
...,...,...,...,...,...,...,...,...,...
36571,03.03.2020,19,37936.79,2020-03-03 19:00:00,35049.21,39283.70,3,3,1
36572,03.03.2020,20,37553.37,2020-03-03 20:00:00,34901.84,38791.40,3,3,1
36573,03.03.2020,21,36581.73,2020-03-03 21:00:00,34225.14,37588.86,3,3,1
36574,03.03.2020,22,35434.36,2020-03-03 22:00:00,33257.06,36290.64,3,3,1


In [12]:
special_day_df["date"]=pd.to_datetime(special_day_df["date"])
special_day_df["date"].dtype

dtype('<M8[ns]')

In [13]:
cons_lagged=pd.merge(cons_lagged,special_day_df,left_on="Datetime",right_on="date",how="left").drop(columns="date").fillna(0)
cons_lagged

,Date,Hour,Consumption,Datetime,Consumption_2_lag,Consumption_7_lag,Day,Month,weekday,is_annelergunu,is_arife,is_babalargunu,is_cocukbayrami,is_cumhuriyet,is_elections,is_examination,is_genclikbayrami,is_holiday,is_iscibayrami,is_kadinlargunu,is_kandil,is_kurbanbayrami,is_kurbanbayramilkgun,is_kurbanoncesigunler,is_okulacilisi,is_okulkapanisi,is_onbestemmuz,is_outlierdays,is_ramazanbayrami,is_ramazanbayramilkgun,is_ramazanbayramisonrasi,is_ramazangunler,is_ramazanilkgun,is_ramazanoncesigunler,is_school_days,is_school_end,is_school_start,is_semester,is_semester_days,is_sevgililergunu,is_single_day_holidays,is_summerTimes,is_summer_break,is_uzunbayram,is_winterTimes,is_winter_break,is_yilbasi,is_yilbasioncesi,is_zaferbayrami
0,07.01.2016,0,28763.95,2016-07-01 00:00:00,29244.38,29590.88,7,1,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,07.01.2016,1,27284.84,2016-07-01 01:00:00,27637.23,27785.43,7,1,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,07.01.2016,2,26321.95,2016-07-01 02:00:00,26335.78,26516.53,7,1,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,07.01.2016,3,25748.49,2016-07-01 03:00:00,25870.23,26091.62,7,1,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,07.01.2016,4,25636.58,2016-07-01 04:00:00,25975.07,25871.54,7,1,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35971,03.03.2020,19,37936.79,2020-03-03 19:00:00,35049.21,39283.70,3,3,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35972,03.03.2020,20,37553.37,2020-03-03 20:00:00,34901.84,38791.40,3,3,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35973,03.03.2020,21,36581.73,2020-03-03 21:00:00,34225.14,37588.86,3,3,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35974,03.03.2020,22,35434.36,2020-03-03 22:00:00,33257.06,36290.64,3,3,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import hstack

cat_features=["Hour","Day","Month","weekday"]
linear_enc=OneHotEncoder()
one_hot_encoded=linear_enc.fit_transform(cons_lagged[cat_features]).toarray()
normal_feats=cons_lagged.columns.difference(cat_features).tolist()
cons_lagged=np.hstack([cons_lagged[normal_feats],one_hot_encoded])
cons_lagged=pd.DataFrame(cons_lagged,columns=normal_feats+np.concatenate([[cat_features[i]+"_"+str(j) for j in cats_]for i,cats_ in enumerate(linear_enc.categories_)]).tolist())

In [15]:
cons_lagged

,Consumption,Consumption_2_lag,Consumption_7_lag,Date,Datetime,is_annelergunu,is_arife,is_babalargunu,is_cocukbayrami,is_cumhuriyet,is_elections,is_examination,is_genclikbayrami,is_holiday,is_iscibayrami,is_kadinlargunu,is_kandil,is_kurbanbayrami,is_kurbanbayramilkgun,is_kurbanoncesigunler,is_okulacilisi,is_okulkapanisi,is_onbestemmuz,is_outlierdays,is_ramazanbayrami,is_ramazanbayramilkgun,is_ramazanbayramisonrasi,is_ramazangunler,is_ramazanilkgun,is_ramazanoncesigunler,is_school_days,is_school_end,is_school_start,is_semester,is_semester_days,is_sevgililergunu,is_single_day_holidays,is_summerTimes,is_summer_break,is_uzunbayram,...,Day_11,Day_12,Day_13,Day_14,Day_15,Day_16,Day_17,Day_18,Day_19,Day_20,Day_21,Day_22,Day_23,Day_24,Day_25,Day_26,Day_27,Day_28,Day_29,Day_30,Day_31,Month_1,Month_2,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6
0,28764,29244.4,29590.9,07.01.2016,2016-07-01 00:00:00,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,27284.8,27637.2,27785.4,07.01.2016,2016-07-01 01:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,26322,26335.8,26516.5,07.01.2016,2016-07-01 02:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,25748.5,25870.2,26091.6,07.01.2016,2016-07-01 03:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,25636.6,25975.1,25871.5,07.01.2016,2016-07-01 04:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35971,37936.8,35049.2,39283.7,03.03.2020,2020-03-03 19:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
35972,37553.4,34901.8,38791.4,03.03.2020,2020-03-03 20:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
35973,36581.7,34225.1,37588.9,03.03.2020,2020-03-03 21:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
35974,35434.4,33257.1,36290.6,03.03.2020,2020-03-03 22:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [0]:
train_df=cons_lagged.loc[cons_lagged["Datetime"]<=pd.to_datetime("2019-03-03 23:00:00")]
valid_df=cons_lagged.loc[cons_lagged["Datetime"]>pd.to_datetime("2019-03-03 23:00:00")]

In [0]:
from sklearn.linear_model import LinearRegression

cons_lagged=cons_lagged.dropna()

cols=train_df.columns.difference(["Date","Consumption","Datetime"]).values

model=LinearRegression()
model.fit(train_df[cols].values,train_df["Consumption"].values.reshape(-1, 1))
preds=model.predict(valid_df[cols].values)

In [0]:
sp_days_index=[i for i,col in enumerate(train_df[cols].columns) if col.startswith("is_")]

In [19]:
cons_lagged=pd.merge(consumption_df,special_day_df,left_on="Datetime",right_on="date",how="left").drop(columns="date").fillna(0)
cons_lagged

,Date,Hour,Consumption,Datetime,is_annelergunu,is_arife,is_babalargunu,is_cocukbayrami,is_cumhuriyet,is_elections,is_examination,is_genclikbayrami,is_holiday,is_iscibayrami,is_kadinlargunu,is_kandil,is_kurbanbayrami,is_kurbanbayramilkgun,is_kurbanoncesigunler,is_okulacilisi,is_okulkapanisi,is_onbestemmuz,is_outlierdays,is_ramazanbayrami,is_ramazanbayramilkgun,is_ramazanbayramisonrasi,is_ramazangunler,is_ramazanilkgun,is_ramazanoncesigunler,is_school_days,is_school_end,is_school_start,is_semester,is_semester_days,is_sevgililergunu,is_single_day_holidays,is_summerTimes,is_summer_break,is_uzunbayram,is_winterTimes,is_winter_break,is_yilbasi,is_yilbasioncesi,is_zaferbayrami
0,31.12.2015,00:00,29590.88,2015-12-31 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,31.12.2015,01:00,27785.43,2015-12-31 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,31.12.2015,02:00,26516.53,2015-12-31 02:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,31.12.2015,03:00,26091.62,2015-12-31 03:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,31.12.2015,04:00,25871.54,2015-12-31 04:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36139,03.03.2020,19:00,37936.79,2020-03-03 19:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36140,03.03.2020,20:00,37553.37,2020-03-03 20:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36141,03.03.2020,21:00,36581.73,2020-03-03 21:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36142,03.03.2020,22:00,35434.36,2020-03-03 22:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
special_days_effect_dict=dict(zip(train_df[cols].columns[sp_days_index].tolist(),model.coef_[0][sp_days_index].tolist()))
for key_ in special_days_effect_dict.keys():
    cons_lagged[key_]=cons_lagged[key_]*special_days_effect_dict[key_]
cons_lagged["Consumption"]=cons_lagged["Consumption"]-cons_lagged[list(special_days_effect_dict.keys())].sum(axis=1)
cons_lagged["Consumption"]

0        29717.447073
1        27785.430000
2        26516.530000
3        26091.620000
4        25871.540000
             ...     
36139    37936.790000
36140    37553.370000
36141    36581.730000
36142    35434.360000
36143    33827.000000
Name: Consumption, Length: 36144, dtype: float64

In [22]:
special_days_effect_dict

{'is_annelergunu': 1570.1706379057157,
 'is_arife': 2163.9549847695807,
 'is_babalargunu': 1028.7726162799893,
 'is_cocukbayrami': 862.004105980193,
 'is_cumhuriyet': 35.00893354476524,
 'is_elections': 1782.918623629318,
 'is_examination': 841.0632325264393,
 'is_genclikbayrami': 65.75945694324312,
 'is_holiday': -2232.1245944855136,
 'is_iscibayrami': 495.27389134834016,
 'is_kadinlargunu': 795.6987001996041,
 'is_kandil': -347.0618080237739,
 'is_kurbanbayrami': -698.7786658495015,
 'is_kurbanbayramilkgun': 254.06226165499834,
 'is_kurbanoncesigunler': -335.69521370919824,
 'is_okulacilisi': 144.86614399353485,
 'is_okulkapanisi': 722.3710198179901,
 'is_onbestemmuz': 633.9873935719804,
 'is_outlierdays': 1571.5881626037094,
 'is_ramazanbayrami': -2970.0792624868463,
 'is_ramazanbayramilkgun': 877.6476695579977,
 'is_ramazanbayramisonrasi': 1507.8635529810595,
 'is_ramazangunler': 424.8422394294769,
 'is_ramazanilkgun': 815.1018472293597,
 'is_ramazanoncesigunler': -37.8317449703754

In [23]:
sorted(cons_lagged[list(special_days_effect_dict.keys())].sum(axis=1).unique())

[-6603.020877104258,
 -6178.178637674781,
 -5725.37320754626,
 -5328.77093033985,
 -5113.891058294632,
 -4667.415494176112,
 -4452.535622130894,
 -4451.123260781852,
 -4331.720280466913,
 -4236.243388736634,
 -4116.840408421695,
 -4077.658018811915,
 -3862.778146766697,
 -3606.0275053135724,
 -3074.7870477982583,
 -2842.590461657287,
 -2739.973706140793,
 -2525.0938340955745,
 -2288.5806373613127,
 -2034.5183757063141,
 -1736.5122338410965,
 -1521.6323617958778,
 -1400.8170201318983,
 -1185.9371480866796,
 -975.9747807024214,
 -953.5056714918649,
 -894.0306220293296,
 -886.9494004574319,
 -819.4953561721409,
 -511.46062636163964,
 -473.6288813912642,
 -464.51618762423277,
 -462.26228707668855,
 -439.1236548285099,
 -431.3566376294773,
 -426.68444265385733,
 -407.01603604577804,
 -400.54140704977857,
 -316.30996054247896,
 -282.76270757457894,
 -247.38241503146998,
 -210.79851987478995,
 -164.39881833786575,
 -137.8965635810441,
 -126.5670733674903,
 -126.56707336748875,
 -97.7859729923

In [0]:
cons_lagged[["Date","Hour","Consumption","Datetime"]].to_csv("drive/My Drive/colab_datas/IE 492/RealTimeConsumption_unholidayized.csv",index=False)

# Burada interpolasyon yapılacak gecen haftaki saatler özelinde 
#### U U U UWU UWAAA UWU https://www.youtube.com/watch?v=0OZyleriwRU